# Delta Neutral Explorer GMX

In [83]:
import numpy as np
import math


class GLP_Hedged_Position():
    aave_target_health_ratio = 1.5
    aave_liquidation_ratio = 0.85

    def __init__(self, deposit, eth_init_weight, btc_init_weight,
                 clk_init_weight, uni_init_weight, usd_init_weight, glp_apr, health_ratio=1.5):
        self.value = deposit
        self.btc_weight = btc_init_weight
        self.eth_weight = eth_init_weight
        self.clk_weight = clk_init_weight
        self.uni_weight = uni_init_weight
        self.usd_weight = usd_init_weight
        self.glp_apr = glp_apr
        self.aave_target_health_ratio = health_ratio

        temp_crypto_exposure = deposit*(1 - usd_init_weight)
        temp_aave_collateral = self.amount2collateral(temp_crypto_exposure)

        scaling_factor = deposit / \
            (deposit + temp_aave_collateral - temp_crypto_exposure)

        self.aave_collateral = temp_aave_collateral*scaling_factor
        self.glp_exposure = deposit * scaling_factor

        self.btc_exposure = self.glp_exposure * self.btc_weight
        self.eth_exposure = self.glp_exposure * self.eth_weight
        self.clk_exposure = self.glp_exposure * self.clk_weight
        self.uni_exposure = self.glp_exposure * self.uni_weight
        self.usd_exposure = self.glp_exposure * self.usd_weight

        self.btc_short = self.btc_exposure
        self.eth_short = self.eth_exposure
        self.clk_short = self.clk_exposure
        self.uni_short = self.uni_exposure

    def amount2collateral(self, crypto_exposure) -> float:
        return crypto_exposure * self.aave_target_health_ratio / self.aave_liquidation_ratio

    def get_short_crypto(self) -> float:
        return self.eth_short + self.btc_short + self.clk_short + self.uni_short

    def get_crypto_exposure(self) -> float:
        return self.eth_exposure + self.btc_exposure + self.clk_exposure + self.uni_exposure

    def daily_return(self) -> float:
        return self.glp_exposure * self.glp_apr / 365


## Sanity Checking Values

In [74]:
# Using Target Ratios
my_position = GLP_Hedged_Position(1000, 0.28, 0.27,
                                  0.01, 0.01, 0.43, 0.3026)

print("Total AAVE Collateral real     : ", my_position.aave_collateral)
print("Sanity Collateral derived      : ", my_position.glp_exposure *
      (1-my_position.usd_weight)*my_position.aave_target_health_ratio/0.85)
print("Sanity Check totals            : ", my_position.glp_exposure + my_position.aave_collateral *
      (1 - my_position.aave_liquidation_ratio/my_position.aave_target_health_ratio))


Total AAVE Collateral real     :  700.5325686194184
Sanity Collateral derived      :  700.5325686194183
Sanity Check totals            :  1000.0


In [91]:
# My real position:
deposit = 9000
my_position = GLP_Hedged_Position(deposit, 0.2425, 0.1564,
                                  0.0232, 0.0058, 0.5719, 0.3030, 1.6)

apr = my_position.daily_return() * 365 / deposit
apy = ((1 + apr/12)**12 - 1) * 100
# Does not include interest paid on AAVE hedge, but it's around 0.8%

print("Initial deposit                : ", "{:.2f}".format(deposit))
print("Total AAVE Collateral          : ","{:.2f}".format( my_position.aave_collateral))
print("Total GLP Exposure             : ", "{:.2f}".format(my_position.glp_exposure))
print("My crypto exposure             : ", "{:.2f}".format(my_position.get_crypto_exposure()))
print("Daily Return                   : ", "{:.2f}".format(my_position.daily_return()))
print("Hedged APY                     : ", "{:.2f}".format(apy))


Initial deposit                :  9000.00
Total AAVE Collateral          :  5264.09
Total GLP Exposure             :  6532.46
My crypto exposure             :  2795.24
Daily Return                   :  5.42
Hedged APY                     :  24.35
